In [ ]:
#Download file from MTA
#Input into a PD Dataframe (e.g. correctly label columns)
#Helper tables: Define MTA station Mapping
#reformat PD into a usable DF - Unit ID, Date-time, station name, entries, exits
#sort on unit-id (will sort by time too). remove duplicates. [possibly find a better method]

#- Which station has the most number of units? ... Penn Station (unique turnstile units for the Week of Feb 2nd)
#- What is the total number of entries & exits across the subway system for February 1, 2013? ...neet to calculate net entries and exits [2hrs]
#- Let’s define the busy-ness as sum of entry & exit count. What station was the busiest on February 1, 2013? What turnstile was the busiest on that date? ... add busyness column. filter for only feb 1. group by station [30 mins]
#- What stations have seen the most usage growth/decline in 2013? [NEEDS clarification - since daily variance is significant] ...need to download whole dataset for 2012 and 2013. (speed up calculation)...sum up net transactions for the year [3hrs]
#- What dates are the least busy? Could you identify days on which stations were not operating at full capacity or closed entirely? ... [2hrs]
#- Bonus:  What hour is the busiest for station CANAL ST in Q1 2013?

#volume by day by station, by turnstile

import pandas as pd
from datetime import datetime


#Questions
#1) What kind of assumptions can I make about the data (e.g. can I assume counts are accurate?)
#2) What does "door open logon lgf man..." mean?
#3) Does question 1 refer to number of units on Feb 1?
#4) I'm making the assumption about day-cutoffs. credit it based on the day the 4 hour cutof ends.

In [ ]:
raw_MTA_data = pd.read_csv("~/Projects/NYC_MTA_DATA_ANALYSIS/data/turnstile_130202.txt")
raw_MTA_data.columns= ["C/A","UNIT","SCP","DATE1","TIME1","DESC1","ENTRIES1","EXITS1","DATE2","TIME2","DESC2","ENTRIES2","EXITS2","DATE3",
"TIME3","DESC3","ENTRIES3","EXITS3","DATE4","TIME4","DESC4","ENTRIES4","EXITS4","DATE5","TIME5","DESC5","ENTRIES5","EXITS5","DATE6",
"TIME6","DESC6","ENTRIES6","EXITS6","DATE7","TIME7","DESC7","ENTRIES7","EXITS7","DATE8","TIME8","DESC8","ENTRIES8","EXITS8"]
print(raw_MTA_data.head())

In [ ]:
#create MTA_turnstiles df... unit, date-time, entries_raw, exits_raw
#Loop through raw dataset. for each row, make an entry in the MTA turnstile data set.
raw_MTA_data_tmp = raw_MTA_data.copy()
raw_MTA_data_tmp["Unit_ID"] = raw_MTA_data_tmp["C/A"] + raw_MTA_data_tmp["UNIT"] + raw_MTA_data_tmp["SCP"]
raw_MTA_data_tmp = raw_MTA_data_tmp.drop(["C/A", "SCP"],axis=1)

def addTimeStamp(vec):
    x = str(vec[0])
    y = str(vec[1])
    try:
        return datetime.strptime(x + " " + y, "%m-%d-%y %H:%M:%S" ).strftime("%y-%m-%d %H:%M:%S")
    except:
        return "NULL"

for i in range(1,8+1):
    raw_MTA_data_tmp["Datetime_" + str(i)] = raw_MTA_data_tmp[["DATE" + str(i), "TIME" + str(i)]].apply(addTimeStamp,axis=1)

#raw_MTA_data_tmp["Date_time_1"] = raw_MTA_data_tmp.apply(lambda x: datetime.strptime(x["DATE1"] + " " + x["TIME1"], "%m-%d-%y %H:%M:%S" ).strftime("%y-%m-%d %H:%M:%S"),axis=1) 
raw_MTA_data_tmp = raw_MTA_data_tmp.drop(["DATE1", "TIME1", "DATE2", "TIME2", "DATE3", "TIME3", "DATE4", "TIME4", "DATE5", "TIME5", "DATE6", "TIME6", "DATE7", "TIME7", "DATE8", "TIME8"],axis=1)
print(raw_MTA_data_tmp.head())






In [ ]:
#add station name & count number of turnstiles per station
remote_station_mapping = pd.read_excel("../data/Remote-Booth-Station.xls")
remote_station_mapping = remote_station_mapping.drop_duplicates(subset=["Remote"]) #assumption only keep the first occuring station name (e.g. 59th st vs. Lexington ave)
remote_station_mapping["UNIT"] = remote_station_mapping["Remote"]

raw_MTA_data_tmp = raw_MTA_data_tmp.merge(remote_station_mapping[["UNIT", "Station"]], on=["UNIT"])

MTA_turnstiles_unique = raw_MTA_data_tmp.drop_duplicates(subset=["Unit_ID"])
stationWithMostTerminals = MTA_turnstiles_unique.groupby(by="Station").agg(["count"])["UNIT"].sort_values(by="count",).iloc[-1,:]
print("Question 1: 'Which station has the most number of units?'")
print(stationWithMostTerminals)
#print(MTA_turnstiles.groupby(by=["Station"]).count())

In [20]:
#pandas magic
#create 8 different datasets identitical
#for each one, pull out the 4 columns you care about...then concatenate.

#1. explode into 8 rows
raw_MTA_data_tmp_x8 = pd.DataFrame()

for i in range(1,8+1):
    raw_MTA_data_tmp_copy = raw_MTA_data_tmp.copy(deep=True)
    raw_MTA_data_tmp_copy["Unit"] = raw_MTA_data_tmp_copy["Unit_ID"]
    raw_MTA_data_tmp_copy["Datetime"] = raw_MTA_data_tmp_copy["DESC" + str(i)]
    raw_MTA_data_tmp_copy["Station"] = raw_MTA_data_tmp_copy["Station"]
    raw_MTA_data_tmp_copy["Cumulative entries"] = raw_MTA_data_tmp_copy["ENTRIES" + str(i)]
    raw_MTA_data_tmp_copy["Cumulative exits"] = raw_MTA_data_tmp_copy["EXITS" + str(i)]
    raw_MTA_data_tmp_copy["Desc"] = raw_MTA_data_tmp_copy["DESC" + str(i)]
    raw_MTA_data_tmp_x8 = pd.concat([raw_MTA_data_tmp_x8,raw_MTA_data_tmp_copy])

raw_MTA_data_tmp = raw_MTA_data_tmp_x8[raw_MTA_data_tmp_x8.columns[-6:]]

print(raw_MTA_data_tmp)

             Station               Unit Datetime  Cumulative entries  \
0              59 ST   A002R05102-00-00  REGULAR           3967826.0   
1              59 ST   A002R05102-00-00  REGULAR           3969685.0   
2              59 ST   A002R05102-00-00  REGULAR           3971424.0   
3              59 ST   A002R05102-00-00  REGULAR           3973086.0   
4              59 ST   A002R05102-00-00  REGULAR           3974913.0   
...              ...                ...      ...                 ...   
30856  RIT-ROOSEVELT  TRAM2R46900-05-01  REGULAR              5554.0   
30857  RIT-ROOSEVELT  TRAM2R46900-05-01  REGULAR              5554.0   
30858  RIT-ROOSEVELT  TRAM2R46900-05-01  REGULAR              5554.0   
30859  RIT-ROOSEVELT  TRAM2R46900-05-01  REGULAR              5554.0   
30860  RIT-ROOSEVELT  TRAM2R46900-05-01      NaN                 NaN   

       Cumulative exits     Desc  
0             1367578.0  REGULAR  
1             1368236.0  REGULAR  
2             1368820.0  REGUL

In [21]:

#MTA_turnstiles = MTA_turnstiles_backup
#sort & make ID the index...
MTA_turnstiles["ID"] = MTA_turnstiles["Unit"] + " " + MTA_turnstiles["Datetime"] 
MTA_turnstiles_sorted = MTA_turnstiles.sort_values(by=["ID"])
#print(len(MTA_turnstiles_sorted))
MTA_turnstiles_sorted = MTA_turnstiles_sorted.drop_duplicates(subset=["ID"])
#print(len(MTA_turnstiles_sorted))

#calculate net entries / exits for the day
MTA_turnstiles_grouped = MTA_turnstiles_sorted.groupby(["Unit"])
MTA_turnstiles_sorted["Net entries"] = MTA_turnstiles_grouped["Cumulative entries"].transform(pd.Series.diff)
MTA_turnstiles_sorted["Net exits"] = MTA_turnstiles_grouped["Cumulative exits"].transform(pd.Series.diff)

#data cleaning (deal with edge cases like negatives, NaN, etc. )
# (MTA_turnstiles_sorted["Net entries"].astype("float64").describe(percentiles=[.1,.25,.5,.75,.9]))
# print(MTA_turnstiles_sorted["Net exits"].describe())
# print('Number of negative entries: %d' %len(MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted['Net entries'] < 0])) #caused by system resets
# print('Number of negative exits: %d' %len(MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted['Net exits'] < 0])) #caused by system resets
# print('Number of NaN entries: %d' %len(MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted['Net entries'].isnull()])) #caused by net zero difference
# print('Number of NaN exits: %d' %len(MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted['Net exits'].isnull()])) #caused by net zero difference

#set outliers to zero
MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted["Net entries"] < 0] = 0
MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted["Net exits"] < 0] = 0
MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted["Net entries"] > 10000] = 0 #assumption: max error threshold
MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted["Net exits"] > 10000] = 0 #assumption: max error threshold
MTA_turnstiles_sorted["Net entries"] = MTA_turnstiles_sorted["Net entries"].fillna(0)
MTA_turnstiles_sorted["Net exits"] = MTA_turnstiles_sorted["Net exits"].fillna(0)

# print('Number of negative entries: %d' %len(MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted['Net entries'] < 0])) #caused by system resets
# print('Number of negative exits: %d' %len(MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted['Net exits'] < 0])) #caused by system resets
# print('Number of NaN entries: %d' %len(MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted['Net entries'].isnull()])) #caused by net zero difference
# print('Number of NaN exits: %d' %len(MTA_turnstiles_sorted.loc[MTA_turnstiles_sorted['Net exits'].isnull()])) #caused by net zero difference



In [22]:
# add days
MTA_turnstiles_sorted["Date"] = pd.to_datetime(MTA_turnstiles_sorted["Datetime"], errors="coerce", yearfirst = True).dt.strftime('%y-%m-%d')
MTA_turnstiles_by_day = MTA_turnstiles_sorted.groupby(by=["Unit", "Date"]).agg({"Net entries": ["sum"]})
MTA_turnstiles_by_day = MTA_turnstiles_by_day.reset_index()
MTA_turnstiles_by_day.columns = MTA_turnstiles_by_day.columns.get_level_values(0)
#print(MTA_turnstiles_by_day)
print("Question 2: What is the total number of entries & exits across the subway system for February 1, 2013?")
print(MTA_turnstiles_by_day.loc[MTA_turnstiles_by_day["Date"] == "13-02-01"].sum()["Net entries"]) #should be about 5M

Question 2: What is the total number of entries & exits across the subway system for February 1, 2013?
5218088.0


In [ ]:
print("Let’s define the busy-ness as sum of entry & exit count. What station was the busiest on February 1, 2013? What turnstile was the busiest on that date?")
MTA_turnstiles_sorted["Busy-ness"] = MTA_turnstiles_sorted["Net entries"] + MTA_turnstiles_sorted["Net exits"] 
MTA_stations_by_day = MTA_turnstiles_sorted.groupby(by=["Station", "Date"]).agg({"Busy-ness": ["sum"]})
MTA_stations_by_day = MTA_stations_by_day.reset_index()
MTA_stations_by_day.columns = MTA_stations_by_day.columns.get_level_values(0)
MTA_stations_by_day = MTA_stations_by_day.loc[MTA_stations_by_day["Date"] == "13-02-01"].sort_values(by="Busy-ness")

busiestStationName = MTA_stations_by_day.iloc[-1]["Station"]
busiestStationTransactions = int(MTA_stations_by_day.iloc[-1]["Busy-ness"])

print("The most busy station is {0} and it had {1} entries/exits".format(busiestStationName, busiestStationTransactions)) #likely  times square, grand central,

In [ ]:
# MTA_turnstiles_by_day.loc[MTA_turnstiles_by_day["Net entries"] > 10000] = 0
# print(MTA_turnstiles_by_day.loc[MTA_turnstiles_by_day["Net entries"] > 10000])

print() 

In [ ]:
MTA_turnstiles["Index column"] = 
MTA_turnstiles["Column 1"]
MTA_turnstiles["Column 2"] = 